# Molecular dynamics of an explicitly solvated small protein

For the simulations in this notebook, we will make use of the crystal structure of the Villin Headpiece subdomain [1YRF](https://www.rcsb.org/structure/1YRF). This is relatively small protein (35 residues) consisting of three alpha helices. This is a prototypical fast folding protein, see [10.1016/j.jmb.2006.03.034](https://doi.org/10.1016/j.jmb.2006.03.034) and [10.1073/pnas.0502495102](https://doi.org/10.1073/pnas.0502495102), and it is therefore a popular benchmark for protein folding molecular dynamics simulations, e.g. [10.1073/pnas.1800690115](https://doi.org/10.1073/pnas.1800690115) (force field accuracy) and [10.1109/SC.2014.9](https://doi.org/10.1109/SC.2014.9) (computational performance).



In [ ]:
import shutil
from sys import stdout

import matplotlib.pyplot as plt
import mdtraj
import nglview
import numpy as np
import pandas
from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *

%matplotlib widget

The PDB file for 1YRF contains more information than we need. It holds several possible states of some residues mixed into one file and OpenMM cannot handle this. Most atomic positions are the same for all these states, but a few have an extra `A`, `B`, `C`, ... just after the atom name. The following function splits such a PDB file into multiples ones. Each of these files is suitable for starting an OpenMM simulation.

In [ ]:
def split_pdb(fn_pdb):
    pos = 16
    groups = {}
    counter = 0
    with open(fn_pdb) as f:
        for line in f:
            if line.startswith("ATOM"):
                state = line[pos]
                line = line[:pos] + " " + line[pos+1:]
                groups.setdefault(state, []).append((counter, line))
                counter += 1
    print("Group keys:", groups.keys())
    for key, lines_group in groups.items():
        if key == " ":
            continue
        lines_both = lines_group + groups[" "]
        lines_both.sort()
        new_fn_pdb = f"{fn_pdb[:-4]}_{key.lower()}.pdb"
        with open(new_fn_pdb, "w") as f:
            for counter, line in lines_both:
                f.write(line)
        print("Written", new_fn_pdb)


split_pdb("1yrf.pdb")

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**
> Inspect the contents of the files `1yrf_a.pdb` and `1yrf_b.pdb`. Verify for at least one atom that these files contain the correct information taken from `1yrf.pdb`.

In [ ]:
pdb = PDBFile('1yrf_a.pdb')
modeller = Modeller(pdb.topology, pdb.positions)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller.addHydrogens(forcefield)
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)
system = forcefield.createSystem(
    modeller.topology, nonbondedMethod=PME, constraints=HBonds)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinIntegrator(temperature, 1/picosecond, 2*femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=100)
positions = simulation.context.getState(getPositions=True).getPositions()
with open('init.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, positions, f)

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

> Which changes to the input structure can be made, for which the force field can still be applied? Try the following:
>
> - Remove an atom from the PDB file.
> - Remove an entire residue from the PDB file.
> - Use an engineered form of the Villin Headpiece, e.g. the Lys24Nle mutant (PDB 1WY3) or the Lys24Nle/Lys29Nle double mutant (PDB 2F4K).
>
> Finally, restore the input to the original one and rerun the above code cell, to have a good starting point for the next cell.
>
> With the methodology shown here, it is not be possible to define custom mutations.

In [ ]:
simulation.reporters = []
simulation.reporters.append(DCDReporter('traj.dcd', 10))
simulation.reporters.append(StateDataReporter(stdout, 100, step=True,
                                              temperature=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter("scalars.csv", 10, step=True, time=True,
                                              potentialEnergy=True, totalEnergy=True, temperature=True))
simulation.step(30000)

# The last line is only needed for Windows users,
# to close the DCD file before it can be opened by nglview.
del simulation

You will need the output files of this simulation for the following notebook. Copy over the files `init.pdb`, `scalars.csv` and `traj.dcd` to the directory `../05_analysis`. This can be done with the following code, or with any file manager that comes with the operating system.

In [ ]:
shutil.copy("init.pdb", "../05_analysis")
shutil.copy("scalars.csv", "../05_analysis")
shutil.copy("traj.dcd", "../05_analysis")

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

> Make an estimate of the computational cost (wall time) on your current hardware to run a sufficiently long simulation to observe spontaneously a protein folding event for this fast folder. The required simulation time is approximately $4 \mathrm{\mu s}$. Would such a calculations be feasible?

In [ ]:
df = pandas.read_csv("scalars.csv")
df.plot(kind='line', x='Time (ps)', y='Potential Energy (kJ/mole)')

In [ ]:
traj = mdtraj.load('traj.dcd', top='init.pdb')
view = nglview.show_mdtraj(traj)
view

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

> Use CHARMM-GUI to create your own mutation of the Villin Headpiece and perform a comparable molecular dynamics simulation.